# **Cockpit Data Model**

In [ ]:
/* ID sequence*/
IF NOT EXISTS
(
SELECT [name]
FROM sys.sequences
WHERE [name] ='IDCode'
)
BEGIN
 
CREATE SEQUENCE [ERM].IDCode  
START WITH 1
    INCREMENT BY 1 ; 
END ELSE BEGIN
ALTER SEQUENCE  [ERM].IDCode RESTART WITH 1
END

# **Table Creation for the Data Model**

## **KPI table (temporal table)**

In [47]:
/* Create KPI table*/
CREATE TABLE [ERM].[KPIs]
(      
    [KPI_ID] [int] NOT NULL PRIMARY KEY,
    [KPI Name] [varchar](200) NOT NULL,
    [KPI Description] [varchar](150) NULL,
    [KPI Group Level 1] [varchar](50) NULL,
    [KPI Group Level 2] [varchar](50) NULL,
    [KPI Group Level 3] [varchar](50) NULL,
    [Date Created] [datetime] DEFAULT GETUTCDATE(),

    /* Escalate missing data after X hours*/
    [Escalate missing data after X Hours] INT NOT NULL DEFAULT (8),

    /* How often is the data to be entered */
    [IsYearly] [TINYINT] DEFAULT 0,
    [IsQuarterly] [TINYINT] DEFAULT 0,
    [IsMonthly] [TINYINT] DEFAULT 0, 
    [IsBiWeekly] [TINYINT] DEFAULT 0 , 
    [IsWeekly] [TINYINT] DEFAULT 0,

    /* Format/Metric of KPI - in which format is the KPI presented*/
    [Format] [varchar] (10) NOT NULL,
        CONSTRAINT [CHECK_Format] CHECK ([Format]= 'Index' OR [Format]= '%' OR [Format]= 'mEUR' OR [Format]= 'EUR' OR [Format]= 'Count' OR [Format]= 'Yes/No'),
    
    /* Aggregation of the KPI - how the KPI should be aggregated across participant area hierachy*/
    [Aggregation] [varchar] (10) NOT NULL,
        CONSTRAINT [CHECK_Aggregation] CHECK ([Aggregation]= 'Sum' OR [Aggregation]= 'Average'),

    /* Desired Direction of the KPI - important to get the logic for KPI traffic light correct*/
    [Desired Direction] [varchar] (10) NOT NULL,
        CONSTRAINT [CHECK_Desired Direction] CHECK ([Desired Direction]= 'Increase' OR [Desired Direction]= 'Decrease'),
    

    /*Temporal table*/
    [ValidFrom] datetime2 GENERATED ALWAYS AS ROW START,
    [ValidTo] datetime2 GENERATED ALWAYS AS ROW END,
    PERIOD FOR SYSTEM_TIME (ValidFrom, ValidTo),

)

WITH (SYSTEM_VERSIONING = ON (HISTORY_TABLE = ERM.KPIsHis)); --temporal table


ALTER TABLE  [ERM].[KPIs]
ADD
DEFAULT NEXT VALUE FOR [ERM].IDCode  
        FOR KPI_ID;  
 
CREATE UNIQUE INDEX UI_KPIName ON [ERM].[KPIs] ([KPI Name] ASC);

Commands completed successfully.

Total execution time: 00:00:00.042

## **Department/Participant Area Table**

In [45]:
-- Participant area table
DROP TABLE IF EXISTS ERM.KPIParticipantArea;  
 
CREATE TABLE ERM.KPIParticipantArea(
    [KPIParticipantArea_ID] [INT]  NOT NULL PRIMARY KEY,
    [VPS] [VARCHAR] (10) NOT NULL ,
    [SVP Area] [VARCHAR] (50) NULL,
    [Module] [VARCHAR] (50) NULL,
    [VPS Responsible]  [VARCHAR] (10) NULL,
    [SVP Area Responsible] [VARCHAR] (10) NULL,
    [Module Responsible] [VARCHAR] (10) NULL,


);

CREATE UNIQUE INDEX UI_VPS_SVPArea_SVPAreaResp_Module_Submodule ON [ERM].[KPIParticipantArea] ([VPS],[SVP Area], [Module] /*,[SubModule]*/);

 
ALTER TABLE  [ERM].[KPIParticipantArea]
ADD
DEFAULT NEXT VALUE FOR [ERM].IDCode  
        FOR KPIParticipantArea_ID;  
 


Commands completed successfully.

Total execution time: 00:00:00.097

## **Actors Table**

In [48]:
--DROP TABLE ERM.Actors

-- Actors table
CREATE TABLE [ERM].[Actors]
( 
    [Actors_ID] [int] NOT NULL PRIMARY KEY,

    /* Primary Key */
    [KPI_ID] [int] NOT  NULL, 
    [KPIParticipantArea_ID] [int] NOT NULL, 
    [Resource_ID] [int] NOT NULL,
    [Role] VARCHAR(20) NOT NULL,

    CONSTRAINT FK_Actors_KPIs FOREIGN KEY (KPI_ID) REFERENCES ERM.KPIs(KPI_ID),-- reference to KPIs table
    CONSTRAINT FK_Actors_KPIParticipantArea FOREIGN KEY (KPIParticipantArea_ID) REFERENCES ERM.KPIParticipantArea(KPIParticipantArea_ID), --reference to ParticipantArea table
    CONSTRAINT FK_Actors_Resource FOREIGN KEY (Resource_ID) REFERENCES ERM.Resource(Resource_ID), --reference to Resource table
            CONSTRAINT [CHECK_Role] CHECK ([Role]= 'VPS Owner' OR [Role]= 'SVP Owner' OR [Role] = 'Module Owner' OR [Role] = 'Submodule Owner' OR [Role]='DataEntry')
)

CREATE UNIQUE INDEX UI_KPIID_KPIParticipantID_Resource_DataEntryUser_ID ON [ERM].[Actors] ([KPI_ID], [KPIParticipantArea_ID], [Resource_ID],[Role]);

ALTER TABLE  [ERM].[Actors]
ADD
DEFAULT NEXT VALUE FOR [ERM].IDCode  
        FOR Actors_ID;  


Commands completed successfully.

Total execution time: 00:00:00.035

## **Target table**

In [49]:
-- Actors table
CREATE TABLE [ERM].[Target]
( 
    [Target_ID] [int] NOT NULL PRIMARY KEY,

    /*Primary key*/
    [KPI_ID] [int] NOT NULL,
    [KPIParticipantArea_ID] [int] NOT NULL, 

    [Green Target] [decimal](18, 2) NOT NULL,
    [Yellow Target] [decimal](18, 2) NOT NULL,

    CONSTRAINT FK_Target_KPIs FOREIGN KEY (KPI_ID) REFERENCES ERM.KPIs(KPI_ID),-- reference to KPIs table
    CONSTRAINT FK_Target_KPIParticipantArea FOREIGN KEY (KPIParticipantArea_ID) REFERENCES ERM.KPIParticipantArea(KPIParticipantArea_ID) --reference to ParticipantArea table
)

CREATE UNIQUE INDEX UI_KPIID_KPIParticipantID_GreenTarget_YellowTarget ON [ERM].[Target] ([KPI_ID], [KPIParticipantArea_ID],[Green Target],[Yellow Target]);

ALTER TABLE  [ERM].[Target]
ADD
DEFAULT NEXT VALUE FOR [ERM].IDCode  
        FOR Target_ID;  

Commands completed successfully.

Total execution time: 00:00:00.035

## **KPI Values table**

In [50]:

/* Create KPI Values Table*/
CREATE TABLE ERM.KPI_Value
(
    [KPIValue_ID] [int]  NOT NULL PRIMARY KEY,
	[KPI_ID] [int] NOT NULL,
    [KPIParticipantArea_ID] [int] NOT NULL ,
	[EnteredbyUser_ID] [int] NOT  NULL, 
	[KPIValueEntryDate_ID]  [int] NOT NULL, --YYYYMMDD
	[KPI Value] [decimal](18, 2) NOT NULL,
    [Comment] [varchar] (200) NULL,
	[Status] [VARCHAR] (30) NOT NULL,

	CONSTRAINT FK_KPIValue_KPIs FOREIGN KEY (KPI_ID) REFERENCES ERM.KPIs(KPI_ID),-- reference to KPIs table
    CONSTRAINT FK_KPIValue_KPIParticipantArea FOREIGN KEY (KPIParticipantArea_ID) REFERENCES ERM.KPIParticipantArea(KPIParticipantArea_ID), --reference to ParticipantArea table
    CONSTRAINT FK_KPIValue_Resource FOREIGN KEY (EnteredbyUser_ID) REFERENCES ERM.Resource(Resource_ID), --reference to Resource table
		CONSTRAINT [CHECK_Status] CHECK ([Status] = 'Missing' OR [Status]='Manual' OR [Status]='Automatic')
)

-- unique index to ensure only one entry per date per participant area per KPI
CREATE UNIQUE INDEX UI_KPIID_KPIParticipantID_KPIValueEntryDate ON [ERM].[KPI_Value] ([KPI_ID], [KPIParticipantArea_ID],[KPIValueEntryDate_ID]);

ALTER TABLE  [ERM].[KPI_Value]
ADD
DEFAULT NEXT VALUE FOR [ERM].IDCode  
        FOR KPIValue_ID;  

Commands completed successfully.

Total execution time: 00:00:00.036

### **KPI Values table: IsCurrent Computed Column**

This column will flag the most recent value for a given KPI.  
The column is created by utilizing a "user defined function".

In [62]:
-- udf for IsCurrent column 
CREATE FUNCTION ERM.udfKPIValueIsCurrent
(
@KPI_ID AS INT,
@KPIParticipantAreaID AS INT,
@KPIValueEntryDate AS INT
)
RETURNS INT
AS
BEGIN
  DECLARE @res AS INT;

  SELECT
    @res = CASE WHEN MAX([KPIValueEntryDate_ID]) OVER(PARTITION BY KPI_ID, KPIParticipantArea_ID)  = @KPIValueEntryDate
        THEN 1 
        ELSE 0
        END
  FROM ERM.KPI_Value
  WHERE
    KPI_ID = @KPI_ID  
    AND KPIParticipantArea_ID = @KPIParticipantAreaID


RETURN @res;
END
GO

Commands completed successfully.

Total execution time: 00:00:00.173

In [52]:
-- add computed column based on udf 
ALTER TABLE ERM.KPI_Value ADD IsCurrent AS ERM.udfKPIValueIsCurrent (KPI_ID,KPIParticipantArea_ID,KPIValueEntryDate_ID)
GO

Commands completed successfully.

Total execution time: 00:00:00.026

# **Stored Procedures**

## **Stored Procedure: New KPI Entry**

This SP will add new KPIs to the datamodel by utilizing multiple merge statements.

In [3]:
    /* Procedure to create new KPIs*/
ALTER PROCEDURE ERM.AddNewKPI
(
    /*KPIs table*/
	 @KPIName [varchar](100),
	 @KPIDescription [varchar](500), --OPTIONAL N/A can be entered
	 @KPIGroupLevel1 [varchar](50),
	 @KPIGroupLevel2 [varchar](50), --OPTIONAL N/A can be entered
    @KPIGroupLevel3 [varchar](50), --OPTIONAL N/A can be entered
    @IsYearly [TINYINT],
    @IsQuarterly [TINYINT],
    @IsMonthly [TINYINT],
    @IsBiWeekly [TINYINT], 
    @IsWeekly [TINYINT],  
    @Format [varchar] (10) ,
    @Aggregation [varchar] (10),
    @DesiredDirection [varchar] (10),

    /*KPIParticipantArea table*/
    @VPS [VARCHAR](50),
    @SVPArea [VARCHAR](50),   --OPTIONAL N/A can be entered
    @Module  [VARCHAR](50),   --OPTIONAL N/A can be entered
    --@SubModule [VARCHAR](50), --OPTIONAL N/A can be entered

    /*Actors table*/
    @DataEntryuserInitials [VARCHAR](10) = 'N/A',     --OPTIONAL only if data entry is not performed by Module or SubModule Owner
    @ProxyDataEntryuserInitials [VARCHAR](10) = 'N/A', --OPTIONAL only if data entry is not performed by Module or SubModule Owner and replacement needed for holidays etc. 

   /*Target table*/
    @GreenTarget [decimal](18, 2),
    @YellowTarget [decimal](18, 2)
)
AS 
BEGIN
     SET NOCOUNT ON;
    /**** Merge w. KPIs table upon new KPI entry ****/
    MERGE ERM.KPIs AS TGT  
        USING (SELECT 
        @KPIName ,
        @KPIDescription,
        @KPIGroupLevel1,
        @KPIGroupLevel2 ,
        @KPIGroupLevel3 ,
        @IsYearly ,
        @IsQuarterly ,
        @IsMonthly ,
        @IsBiWeekly, 
        @IsWeekly,
        @Format,
        @Aggregation,
        @DesiredDirection) 
        AS SRC (KPIName , KPIDescription, KPIGroupLevel1,KPIGroupLevel2 ,KPIGroupLevel3 ,IsYearly ,IsQuarterly ,IsMonthly ,IsBiWeekly, IsWeekly, [Format], Aggregation, DesiredDirection )  
        ON TGT.[KPI Name] = SRC.KPIName
        WHEN MATCHED
            THEN UPDATE SET 
                        TGT.[KPI Description] = SRC.KPIDescription,
                        TGT.[KPI Group Level 1] = SRC.KPIGroupLevel1,
                        TGT.[KPI Group Level 2] = SRC.KPIGroupLevel2,
                        TGT.[KPI Group Level 3] = SRC.KPIGroupLevel3,
                        TGT.IsYearly = SRC.IsYearly,
                        TGT.IsQuarterly = SRC.IsQuarterly,
                        TGT.IsMonthly = SRC.IsMonthly,
                        TGT.IsBiWeekly = SRC.IsBiWeekly,
                        TGT.IsWeekly = SRC.IsWeekly,
                        TGT.[Format] = SRC.Format,
                        TGT.[Aggregation] = SRC.Aggregation,
                        TGT.[Desired Direction] = SRC.DesiredDirection
        WHEN NOT MATCHED 
            THEN INSERT ([KPI Name], [KPI Description],[KPI Group Level 1],[KPI Group Level 2],[KPI Group Level 3],[IsYearly],[IsQuarterly], [IsMonthly], [IsBiWeekly], [IsWeekly], [Format], [Aggregation], [Desired Direction])
            VALUES      (SRC.KPIName,SRC.KPIDescription,SRC.KPIGroupLevel1,SRC.KPIGroupLevel2 ,SRC.KPIGroupLevel3 ,SRC.IsYearly ,SRC.IsQuarterly ,SRC.IsMonthly ,SRC.IsBiWeekly, SRC.IsWeekly, SRC.Format,SRC.Aggregation,SRC.DesiredDirection );



    /**** Merge w. Participant Area table upon new KPI entry ****/
    MERGE ERM.KPIParticipantArea AS TGT  
        USING (SELECT 
        @VPS
        ,@SVPArea
        ,@Module
        )
        AS SRC (VPS,SVPArea, Module)  
        ON TGT.[VPS] = SRC.VPS
        AND TGT.[SVP Area] = SRC.SVPArea
        AND TGT.[Module] = SRC.Module
        WHEN NOT MATCHED 
            THEN INSERT ([VPS], [SVP Area] , [Module] )
            VALUES      (SRC.VPS, SRC.SVPArea, SRC.Module);


    /**** Merge w. Actor table upon new KPI entry for SVP Owner ****/
        -- to get KPI_ID 
     DECLARE @KPIID INT = NULL
     SET @KPIID = 
     (SELECT KPI_ID
     FROM ERM.KPIs AS KPI
     WHERE KPI.[KPI Name]=@KPIName)

        -- to get ParticipantAreaID
     DECLARE @KPIParticipantAreaID INT = NULL
     SET @KPIParticipantAreaID = 
     (SELECT [KPIParticipantArea_ID]
     FROM ERM.KPIParticipantArea AS PA
     WHERE PA.[VPS] = @VPS
     AND PA.[SVP Area]=@SVPArea
     AND PA.[Module] = @Module) 

     -- to get ResourceID SVPOwner
    IF @SVPArea <> 'N/A' 
      BEGIN
               -- Initials of SVP Owner are stored alongside Module in KPIParticipantArea
            DECLARE @SVPOwnerInitials [VARCHAR](10) = NULL
            SET @SVPOwnerInitials = 
            (SELECT [SVP Area Responsible]
            FROM ERM.KPIParticipantArea AS PA
            WHERE PA.KPIParticipantArea_ID = @KPIParticipantAreaID)
            
               -- Now, get the resource ID associated with Initials
            DECLARE @SVPOwnerID INT = NULL
            SET  @SVPOwnerID = 
            (SELECT Resource_ID
            FROM ERM.Resource AS RE
            WHERE RE.[Initials]=@SVPOwnerInitials)
            
            -- Role as SVPOwner
            DECLARE @RoleSVPOwner VARCHAR(20) = NULL
            SET  @RoleSVPOwner = 'SVP Owner'
            
            --Merge
            MERGE ERM.Actors AS TGT  
            USING (SELECT @KPIID, @KPIParticipantAreaID, @SVPOwnerID, @RoleSVPOwner) AS SRC (KPI_ID, KPIParticipantArea_ID, Resource_ID, Role)  
               ON  TGT.[KPI_ID] = SRC.KPI_ID
               AND TGT.[KPIParticipantArea_ID] = SRC.KPIParticipantArea_ID
               AND TGT.[Resource_ID] = SRC.Resource_ID
               AND TGT.[Role] = SRC.Role
            WHEN NOT MATCHED 
               THEN INSERT (KPI_ID     , KPIParticipantArea_ID     , Resource_ID       , Role    )
               VALUES      (SRC.KPI_ID , SRC.KPIParticipantArea_ID , SRC.Resource_ID   , SRC.Role);
      END


     /**** Merge w. Actor table upon new KPI entry for Module Owner ****/
    IF @Module <> 'N/A' 
      BEGIN
            -- to get ResourceID ModuleOwner
               -- Initlas of Responsibles are stored alongside Module in KPIParticipantArea
            DECLARE @ModuleOwnerInitials [VARCHAR](10) = NULL
            SET @ModuleOwnerInitials = 
            (SELECT [Module Responsible]
            FROM ERM.KPIParticipantArea AS PA
            WHERE PA.KPIParticipantArea_ID = @KPIParticipantAreaID)

            -- to get ResourceID ModuleOwner
            DECLARE @ModuleOwnerID INT = NULL
            SET  @ModuleOwnerID = 
            (SELECT Resource_ID
            FROM ERM.Resource AS RE
            WHERE RE.[Initials]= @ModuleOwnerInitials)

            -- Role as SVPOwner
            DECLARE @RoleModuleOwner VARCHAR(20) = NULL
            SET  @RoleModuleOwner = 'Module Owner'

            --Merge
            MERGE ERM.Actors AS TGT  
            USING (SELECT @KPIID, @KPIParticipantAreaID, @ModuleOwnerID, @RoleModuleOwner) AS SRC (KPI_ID, KPIParticipantArea_ID, Resource_ID, Role)  
               ON  TGT.[KPI_ID] = SRC.KPI_ID
               AND TGT.[KPIParticipantArea_ID] = SRC.KPIParticipantArea_ID
               AND TGT.[Resource_ID] = SRC.Resource_ID
               AND TGT.[Role] = SRC.Role
            WHEN NOT MATCHED 
               THEN INSERT (KPI_ID     , KPIParticipantArea_ID     , Resource_ID       , Role    )
               VALUES      (SRC.KPI_ID , SRC.KPIParticipantArea_ID , SRC.Resource_ID   , SRC.Role);
      END


    /**** Merge w. Actor table upon new KPI entry for DataEntry ****/
      -- Role as DataEntry
      DECLARE @RoleDataEntryUser VARCHAR(20) = NULL
      SET  @RoleDataEntryUser = 'DataEntry'
   
   
    IF @DataEntryuserInitials  <> 'N/A'
      BEGIN
            -- to get ResourceID Data Enrty user
            DECLARE @DataEntryUserID INT = NULL
            SET  @DataEntryUserID = 
            (SELECT Resource_ID
            FROM ERM.Resource AS RE
            WHERE RE.[Initials]=@DataEntryuserInitials)

               --Merge
            MERGE ERM.Actors AS TGT  
            USING (SELECT @KPIID, @KPIParticipantAreaID, @DataEntryUserID, @RoleDataEntryUser) AS SRC (KPI_ID, KPIParticipantArea_ID, Resource_ID, Role)  
            ON  TGT.[KPI_ID] = SRC.KPI_ID
            AND TGT.[KPIParticipantArea_ID] = SRC.KPIParticipantArea_ID
            AND TGT.[Resource_ID] = SRC.Resource_ID
            AND TGT.[Role] = SRC.Role
            WHEN NOT MATCHED 
               THEN INSERT (KPI_ID     , KPIParticipantArea_ID     , Resource_ID       , Role)
               VALUES      (SRC.KPI_ID , SRC.KPIParticipantArea_ID , SRC.Resource_ID   , SRC.Role);
      END


  /**** Merge w. Actor table upon new KPI entry for ProxyDataEntry -- for a possible second user that fills in during holiday season sickness etc ****/
   IF @ProxyDataEntryuserInitials <> 'N/A'
      BEGIN
            -- to get ResourceID Data Enrty user
            DECLARE @ProxyDataEntryUserID INT = NULL
            SET  @ProxyDataEntryUserID = 
            (SELECT Resource_ID
            FROM ERM.Resource AS RE
            WHERE RE.[Initials]=@ProxyDataEntryuserInitials)

               --Merge
            MERGE ERM.Actors AS TGT  
            USING (SELECT @KPIID, @KPIParticipantAreaID, @ProxyDataEntryUserID, @RoleDataEntryUser) AS SRC (KPI_ID, KPIParticipantArea_ID, Resource_ID, Role)  
            ON  TGT.[KPI_ID] = SRC.KPI_ID
            AND TGT.[KPIParticipantArea_ID] = SRC.KPIParticipantArea_ID
            AND TGT.[Resource_ID] = SRC.Resource_ID
            AND TGT.[Role] = SRC.Role
            WHEN NOT MATCHED 
               THEN INSERT (KPI_ID     , KPIParticipantArea_ID     , Resource_ID       , Role)
               VALUES      (SRC.KPI_ID , SRC.KPIParticipantArea_ID , SRC.Resource_ID   , SRC.Role);
      END 


 /**** Merge w. Target table upon new KPI entry ****/
    MERGE ERM.Target AS TGT  
        USING (SELECT 
        @KPIID,
        @KPIParticipantAreaID, 
        @GreenTarget,
        @YellowTarget
        )   AS SRC (KPI_ID, KPIParticipantArea_ID, GreenTarget, YellowTarget)  
        ON  TGT.[KPI_ID] = SRC.KPI_ID
        AND TGT.[KPIParticipantArea_ID] = SRC.KPIParticipantArea_ID
        AND TGT.[Green Target] = SRC.GreenTarget
        AND TGT.[Yellow Target] = SRC.YellowTarget
        WHEN NOT MATCHED 
            THEN INSERT (KPI_ID, KPIParticipantArea_ID, [Green Target], [Yellow Target])
            VALUES      (KPI_ID, KPIParticipantArea_ID,  GreenTarget,    YellowTarget);

      
    /**** Merge w. KPI_Value table for first record in KPI_Value ****/
     DECLARE @EnteredByUserID INT = NULL 
     SET  @EnteredByUserID = 
     (SELECT Resource_ID
     FROM ERM.Resource AS RE
     WHERE RE.[Initials] = 'System')

        -- get DateID
    DECLARE @DateCreated DATETIME
    SET @DateCreated = GETDATE()
        --Merge
    MERGE ERM.KPI_Value AS TGT
    USING (SELECT 
            @KPIID,
            @KPIParticipantAreaID, 
            @EnteredByUserID, 
            CAST(CONVERT(VARCHAR,@DateCreated,112) AS INT) KPIValueEntryDate_ID -- YYYYMMDD 
         )   AS SRC (KPI_ID, KPIParticipantArea_ID, EnteredbyUser_ID, KPIValueEntryDate_ID)
    ON TGT.[KPI_ID] = SRC.KPI_ID
    AND TGT.[KPIParticipantArea_ID] = SRC.KPIParticipantArea_ID
    AND TGT.KPIValueEntryDate_ID = SRC.KPIValueEntryDate_ID
        WHEN NOT MATCHED THEN INSERT (KPI_ID,KPIParticipantArea_ID, EnteredbyUser_ID    ,KPIValueEntryDate_ID,[KPI Value],[Comment],[Status])
        VALUES                       (KPI_ID,KPIParticipantArea_ID, EnteredbyUser_ID    ,KPIValueEntryDate_ID,-1.00 ,'Missing'       ,'Missing');
END

Commands completed successfully.

Total execution time: 00:00:00.061

## **Stored Procedure: add new records on chosen time interval**

Upon execution, this SP will generate new records based on the given interval selected for a KPI.  
The SP will be executed automatically by utilizing _Microsoft Power Automate._

In [12]:
ALTER PROCEDURE [ERM].[AddNewEmptyRecordsToKPIValue]
(
@RUN [INT]
)
AS 
    BEGIN
/* Weekly means Sunday 23:59 */
/* Monthly is end of month at 23:59 */

/* Assume the above is here in all subsequent code block. */
        SET DATEFIRST  1, -- 1 = Monday, 7 = Sunday
            DATEFORMAT mdy, 
            LANGUAGE   US_ENGLISH;

        /* From which date are we generating */
        DECLARE @StartDate DATE;
        SELECT @StartDate  =MIN(DATEADD(day, DATEDIFF(day, 6,[Date Created]-1) /7*7 + 7, 6)) FROM [ERM].[KPIs]

        /* What is our final date */
        DECLARE @CutoffDate date = DATEADD(yy, DATEDIFF(yy, 0, GETDATE()) + 1, -1);

        /* Generate date range */
        ;WITH dates(n) AS 
        (
        SELECT 0 UNION ALL SELECT n + 1 FROM dates
        WHERE n < DATEDIFF(DAY, @StartDate , @CutoffDate)
        )
        ,DatesFlattened AS 
        (
        SELECT DATEADD(DAY,n,@StartDate) Date
        FROM dates
        )
        ,Dates_WeeklyBiWeeklyMonthlyYearly AS
        (

        SELECT Date
                ,KPI.[KPI_ID]
                ,RES.[Resource_ID]
                ,PA.[KPIParticipantArea_ID]
        FROM DatesFlattened Weekly
        INNER JOIN [ERM].[KPIs] KPI ON /* Yearly */

                                        (
                                        KPI.IsYearly=1
                                        AND Weekly.Date=DATEADD(yy, DATEDIFF(yy, 0, weekly.DATE) + 1, -1) /* Take end of Year*/
                                        AND Weekly.Date>DATEADD(day, DATEDIFF(day, 6,KPI.[Date Created]-1) /7*7 + 7, 6)
                                                AND Weekly.Date <=DATEADD(yy,-1,DATEADD(yy, DATEDIFF(yy, 0, GETDATE()) + 1, -1) )/* Stop at current YEAR*/
                                        )
                                        OR
                                        /* Monthly */
                                        (KPI.IsMonthly=1
                                        AND Weekly.Date=EOMONTH(Weekly.Date) /* Take end of Month */
                                        AND Weekly.Date>DATEADD(day, DATEDIFF(day, 6,KPI.[Date Created]-1) /7*7 + 7, 6)
                                                AND Weekly.Date <=DATEADD(MONTH,-1,EOMONTH(GETDATE())) /* Stop at current month */
                                        )
                                        OR
                                        /* Biweekly */
                                        (
                                        KPI.IsBiWeekly=1 
                                        AND Weekly.Date>DATEADD(day, DATEDIFF(day, 6,KPI.[Date Created]-1) /7*7 + 7, 6)
                                        AND Weekly.Date=DATEADD(dd, 7-(DATEPART(dw, Date)), Date) /* Take Sunday */
                                        AND DATEDIFF(DAY,Weekly.date,DATEADD(day, DATEDIFF(day, 6,KPI.[Date Created]-1) /7*7 + 7, 6))  % 14=0 /* Take Biweekly */
                                        AND Date<=DATEADD(WEEK, 0, DATEADD(DAY, 1-DATEPART(WEEKDAY, GETDATE()), DATEDIFF(dd, 0, GETDATE()))) /* Stop at Sunday of the current week*/
                                        )
                                        OR
                                        /* Weekly */
                                        (
                                        KPI.IsWeekly=1 
                                        AND Weekly.Date>DATEADD(day, DATEDIFF(day, 6,KPI.[Date Created]-1) /7*7 + 7, 6)
                                        AND Date=DATEADD(dd, 7-(DATEPART(dw, Date)), Date) /* Take Sunday */
                                        AND Date<=DATEADD(WEEK, 0, DATEADD(DAY, 1-DATEPART(WEEKDAY, GETDATE()), DATEDIFF(dd, 0, GETDATE())))  /* Stop at Sunday of the current week*/
                                        )
										OR
                                        /* Quarter*/
                                        (
                                        KPI.IsQuarterly=1 
                                        AND Weekly.Date=dateadd(qq,1,dateadd(qq, DateDiff(qq, 0, date), -1)) /* Take end of Q*/
                                        AND Weekly.Date>DATEADD(day, DATEDIFF(day, 6,KPI.[Date Created]-1) /7*7 + 7, 6)
                                                AND Weekly.Date <=dateadd(qq, DateDiff(qq, 0, GETDATE()), -1) /* Stop at current Q*/
                                        )
        INNER JOIN [ERM].[Resource] RES ON RES.Initials='System'
        INNER JOIN [ERM].[Actors] ACT ON ACT.KPI_ID=KPI.KPI_ID
        INNER JOIN [ERM].[KPIParticipantArea] PA ON ACT.KPIParticipantArea_ID=PA.KPIParticipantArea_ID 
        )
        MERGE ERM.KPI_Value AS TARGET
        USING (
                SELECT DISTINCT CAST(CONVERT(VARCHAR,CAST(DATE AS DATETIME),112) AS INT) DATE_ID -- YYYYMMDD
                                ,KPI_ID
                                ,Resource_ID
                                ,[KPIParticipantArea_ID]
                FROM Dates_WeeklyBiWeeklyMonthlyYearly 
            ) AS SOURCE (Date_ID, KPI_ID,Resource_ID,KPIParticipantArea_ID)
        ON (TARGET.KPI_ID = SOURCE.KPI_ID 
            AND TARGET.KPIValueEntryDate_ID = SOURCE.DATE_ID 
                    AND TARGET.[KPIParticipantArea_ID]=SOURCE.[KPIParticipantArea_ID])
        WHEN NOT MATCHED THEN INSERT (KPI_ID,KPIParticipantArea_ID,EnteredbyUser_ID,KPIValueEntryDate_ID,[KPI Value],[Comment],[Status])
                            VALUES (KPI_ID,KPIParticipantArea_ID,Resource_ID      ,Date_ID             ,-1.00       ,'Missing','Missing')
        OPTION (MAXRECURSION 0);
    END;

Commands completed successfully.

Total execution time: 00:00:00.357

## **Stored Procedure: KPI Values Entry**

This SP enables users to update values for a selected KPI on a given date by utilizing multiple merge statements. 

The SP also incorporates a security mechanism - only users with the correct access will be allowed to edit the selected KPI Values.

The SP only allows editing of KPI Values that are already present i.e. the associated records that are generated by the previous SP.

In [22]:
/* Procedure to update KPI Values */
ALTER PROCEDURE ERM.UpdateKPIValues
(
@KPIName [varchar](200),
@VPS [VARCHAR](50),
@SVPArea [VARCHAR](50),          --OPTIONAL N/A can be entered
@Module  [VARCHAR](50),          --OPTIONAL N/A can be entered
@KPIEntryUserEmail [VARCHAR](30),
@KPIValueEntryDate [INT], 
@KPIValue [DECIMAL](18,2),
@Comment[VARCHAR](200) = NULL,
@Status [VARCHAR](10)
)
AS
    BEGIN
    -- get KPI_ID
     DECLARE @KPIID INT = NULL
     SET @KPIID = 
     (SELECT KPI_ID
     FROM ERM.KPIs AS KPI
     WHERE KPI.[KPI Name]=@KPIName)

    -- get KPI area ID
     DECLARE @KPIParticipantAreaID INT = NULL
     SET @KPIParticipantAreaID = 
     (SELECT [KPIParticipantArea_ID]
     FROM ERM.KPIParticipantArea AS PA
     WHERE PA.[VPS] = @VPS
     AND PA.[SVP Area]=@SVPArea
     AND PA.[Module] = @Module)

    -- get Resource ID 
     DECLARE @ResourceID INT = NULL
     SET  @ResourceID = 
     (SELECT Resource_ID
     FROM ERM.Resource AS RE
     WHERE RE.[Email]=@KPIEntryUserEmail)

    /*Checks if User that tries to update, has Edit rights*/
    DECLARE @EditRightsCheck INT = NULL
    SET @EditRightsCheck =  CASE WHEN EXISTS(SELECT 1
                                    FROM  ERM.Actors AC
                                    WHERE AC.KPI_ID = @KPIID
                                    AND AC.KPIParticipantArea_ID = @KPIParticipantAreaID
                                    AND AC.Resource_ID = @ResourceID)
                                THEN 1  ELSE 0
                            END 
    IF @EditRightsCheck = 0
    BEGIN   
        RAISERROR('Error: You do not have the rights to edit this KPI',16,1);
    END

        /*Checks if date is valid*/
    DECLARE @DateCheck INT = NULL
    SET @DateCheck =  CASE WHEN EXISTS(SELECT 1
                                    FROM  ERM.KPI_Value AC
                                    WHERE AC.KPI_ID = @KPIID
                                    AND AC.KPIParticipantArea_ID = @KPIParticipantAreaID
                                    AND AC.KPIValueEntryDate_ID = @KPIValueEntryDate)
                                THEN 1  ELSE 0
                            END 
    IF @DateCheck = 0
    BEGIN   
        RAISERROR('Error: The selected date is invalid',16,1);
    END

        --Merge 
        MERGE ERM.KPI_Value AS TGT
        USING (
        SELECT @KPIID, @KPIParticipantAreaID, @ResourceID, @KPIValueEntryDate, @KPIValue, @Comment, @Status
        )
        AS SRC (KPI_ID, KPIParticipantArea_ID, EnteredbyUser_ID, KPIValueEntryDate_ID, KPIValue,Comment, Status)
        ON TGT.KPI_ID = SRC.KPI_ID 
        AND TGT.KPIParticipantArea_ID = SRC.KPIParticipantArea_ID
        AND TGT.KPIValueEntryDate_ID = SRC.KPIValueEntryDate_ID
        WHEN MATCHED 
            THEN UPDATE SET
            TGT.[KPI Value] = SRC.KPIValue,
            TGT.[Comment] = SRC.Comment,
            TGT.[Status] = SRC.Status;

    END

Commands completed successfully.

Total execution time: 00:00:00.230

# **Example data**

In [79]:
-- add some example KPIs
EXECUTE ERM.AddNewKPI '1.1 Revenue','N/A','Balance Sheet','N/A','N/A',0,1,0,0,0,'mEUR','Sum','Increase','Firm level','N/A','N/A','HOHOHO','HEHEHE','200','180'
EXECUTE ERM.AddNewKPI '1.2 EBIT','N/A','Balance Sheet','N/A','N/A',0,1,0,0,0,'%','Average','Increase','Firm level','N/A','N/A','HOHOHO','HEHEHE','8','6'
EXECUTE ERM.AddNewKPI '1.3 Free Cash flow','N/A','Balance Sheet','N/A','N/A',0,1,0,0,0,'mEUR','Sum','Increase','Firm level','N/A','N/A','HOHOHO','HEHEHE','50','10'
EXECUTE ERM.AddNewKPI '2.1 Active Customer Complaints','N/A','Customer','Customer Support','N/A',0,0,0,0,1,'Count','Sum','Decrease','Firm level','Department level','N/A','HOHOHO','HEHEHE','10','15'
EXECUTE ERM.AddNewKPI '2.1 Active Customer Complaints','N/A','Customer','Customer Support','N/A',0,0,0,0,1,'Count','Sum','Decrease','Firm level','Department level','N/A','HOHOHO','HEHEHE','10','15'
EXECUTE ERM.AddNewKPI '2.1 Active Customer Complaints','N/A','Customer','Customer Support','N/A',0,0,0,0,1,'Count','Sum','Decrease','Firm level','Department level','N/A','HOHOHO','HEHEHE','10','15'
EXECUTE ERM.AddNewKPI '2.1 Active Customer Complaints','N/A','Customer','Customer Support','N/A',0,0,0,0,1,'Count','Sum','Decrease','Firm level','Department level','N/A','HOHOHO','HEHEHE','10','15'
EXECUTE ERM.AddNewKPI '2.1 Active Customer Complaints','N/A','Customer','Customer Support','N/A',0,0,0,0,1,'Count','Sum','Decrease','Firm level','Department level','N/A','HOHOHO','HEHEHE','10','15'
EXECUTE ERM.AddNewKPI '2.2 Customer Satisfaction Score','N/A','Customer','Customer Support','N/A',0,0,0,0,1,'%','Sum','Increase','Firm level','Department level','N/A','HOHOHO','HEHEHE','80','75'
EXECUTE ERM.AddNewKPI '2.2 Customer Satisfaction Score','N/A','Customer','Customer Support','N/A',0,0,0,0,1,'%','Sum','Increase','Firm level','Department level','N/A','HOHOHO','HEHEHE','80','75'
EXECUTE ERM.AddNewKPI '2.2 Customer Satisfaction Score','N/A','Customer','Customer Support','N/A',0,0,0,0,1,'%','Sum','Increase','Firm level','Department level','N/A','HOHOHO','HEHEHE','80','75'
EXECUTE ERM.AddNewKPI '2.2 Customer Satisfaction Score','N/A','Customer','Customer Support','N/A',0,0,0,0,1,'%','Sum','Increase','Firm level','Department level','N/A','HOHOHO','HEHEHE','80','75'

Commands completed successfully.

Total execution time: 00:00:02.174